In [286]:
# Imports
import numpy as np
import random

In [287]:
# Set-up

# Parameters
genome_size = 16
max_group_size = 6
starting_group_size = 5
group_number = 4
mutation_rate = 0.5
group_split_rate = 1

# Create initial population
initial_population = np.random.randint(0, 2, size=(group_number, max_group_size, genome_size), dtype=np.int8)
population = initial_population.copy()
display(population)

# Keep track of group sizes
group_sizes = np.full((group_number), starting_group_size, dtype = np.int8)
display(group_sizes)

array([[[1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0]],

       [[1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0],
        [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
        [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
        [1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1],
        [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1]],

       [[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
        [0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
        [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1],
        [0

array([5, 5, 5, 5], dtype=int8)

In [288]:
# Fitness table

# Take population array and calculate the fitness for each indiviual in the initial population

# Calculate the means of the genomes
fitness_table = np.mean(population, axis=2, keepdims=True)

# Set 'empty' slots to 0
for i, size in enumerate(group_sizes):
    fitness_table[i, size:] = 0

fitness_array = fitness_table.flatten()
display(fitness_array)


# If a group splits or a individual is born, recalculate the fitnesses in that/those group(s)
# Than update this table 
# To not calculate the fitnesses of each individuals each timestep but only SOME individuals SOMEtimes

array([0.375 , 0.375 , 0.1875, 0.6875, 0.5625, 0.    , 0.4375, 0.6875,
       0.6875, 0.4375, 0.5   , 0.    , 0.5625, 0.5   , 0.5625, 0.3125,
       0.625 , 0.    , 0.5625, 0.4375, 0.5625, 0.625 , 0.5625, 0.    ])

In [302]:
def replaceMemberEvent(pop_input, group_id_input, newborn_genome_input):
    
    # Replace one unlucky member with newborn
    unlucky_index = np.random.choice(max_group_size)
    pop_input[group_id_input, [unlucky_index]] = newborn_genome_input
    print(f"Individual {unlucky_index} in group {group_id_input} died")
    # Recalculate fitness 

    new_fitness = np.mean(population[group_id_input], axis=1, keepdims= False)
    new_fitness[group_sizes[group_id_input]:] = 0 
    # could be more efficent to only calulate it for active spots instead of making inactive 0 after

    fitness_array[group_id_input*max_group_size : group_id_input*max_group_size+max_group_size] = new_fitness
    print('Current fitness array:')
    display(fitness_array)

    return pop_input

def groupSplitEvent(pop_input, group_id_input, newborn_genome_input):
    # All groups except the current one
    left_over_groups = [i for i in range(group_number) if i != group_id_input]
    unlucky_group_index = np.random.choice(left_over_groups)
    # Set group size of unlucky group to 0
    group_sizes[unlucky_group_index] = 0

    # Boolean mask to split the group in a random way
    move_mask = np.random.choice([True, False], size = max_group_size, p = [0.5, 0.5])
        
    if not np.any(move_mask): # If all false, select 1 random to be true
        move_mask[np.random.randint(0, max_group_size)] = True
    elif np.all(move_mask):   # Vice versa
        move_mask[np.random.randint(0, max_group_size)] = False
            
    # Move those marked 'True' to new group
    moving_members = pop_input[group_id_input, move_mask, :]
    num_moving = len(moving_members)
    # Place them at the top of the new group
    pop_input[unlucky_group_index, 0:num_moving, :] = moving_members
    # Set group size to new amount
    group_sizes[unlucky_group_index] = num_moving
        
    # Move those marked 'False' to top of old group
    staying_members = pop_input[group_id_input, ~move_mask, :]
    num_staying = len(staying_members)
    # Place on top
    pop_input[group_id_input, 0:num_staying, :] = staying_members
    # Set group size to new amount
    group_sizes[group_id_input] = num_staying

    # Debug
    print(f"Group {group_id_input} split and replaced group {unlucky_group_index}.")
    print(f"{num_moving} individuals moved. {num_staying} stayed.")
    
    # Add newborn to old group
    # Place gemone copy below the taken spaces, overwriting the 'garbage' data
    pop_input[group_id_input, group_sizes[group_id_input]] = newborn_genome_input
    print(f"Reproduced member in group {group_id_input} at slot {group_sizes[group_id_input]}")

    # Increase group size so new individual is considered as 'actual' data
    group_sizes[group_id_input] += 1

    # Recalculate fitness for both groups

    fitness_old_group = np.mean(population[group_id_input], axis=1, keepdims= False)
    fitness_old_group[group_sizes[group_id_input]:] = 0
    fitness_new_group = np.mean(population[unlucky_group_index], axis=1, keepdims= False)
    fitness_new_group[group_sizes[unlucky_group_index]:] = 0
    fitness_array[group_id_input*max_group_size : group_id_input*max_group_size+max_group_size] = fitness_old_group
    fitness_array[unlucky_group_index*max_group_size: unlucky_group_index*max_group_size+max_group_size] = fitness_new_group   
   
    return pop_input

def birthEvent(pop_input, group_id_input, newborn_genome_input):

    # Place gemone copy below the taken spaces, overwriting the 'garbage' data
    pop_input[group_id_input, group_sizes[group_id_input]] = newborn_genome_input
    print(f"Reproduced member in group {group_id_input} at slot {group_sizes[group_id_input]}")

    # Increase group size so new individual is considered as 'actual' data
    group_sizes[group_id_input] += 1

    # Recalculate fitness
    new_fitness = np.mean(population[group_id_input], axis=1, keepdims= False)
    new_fitness[group_sizes[group_id_input]:] = 0 
    fitness_array[group_id_input*max_group_size : group_id_input*max_group_size+max_group_size] = new_fitness
    # could be more efficent to only calulate it for active spots instead of making inactive 0 after

    return pop_input

In [ ]:
# Reproduction function

def reproductionEvent(pop_input):
    # Select a random individual from fitness array based on weight
    probabilities = fitness_array / np.sum(fitness_array)
    selected_index = np.random.choice(len(fitness_array), p=probabilities)
    # Get group and member id
    group_id = selected_index//max_group_size
    member_id = selected_index%max_group_size

    print(f'Selected member {member_id} of group {group_id}')

    # Get genome of selected individual
    selected_genome = pop_input[group_id, member_id]

    # Mutate genome here first

    # Check if group is full
    if group_sizes[group_id] == max_group_size:
        # Either split or replace
        if random.random() > group_split_rate:
            replaceMemberEvent(pop_input, group_id, selected_genome)
        else:
            groupSplitEvent(pop_input, group_id, selected_genome)
    # Else just add new member to group
    else:
        birthEvent(pop_input, group_id, selected_genome)

In [512]:
reproductionEvent(population)


Selected member 1 of group 3
Reproduced member in group 3 at slot 3
